<a href="https://colab.research.google.com/github/atharvas/dirp_neurosymbolic/blob/master/Week1_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
